In [ ]:
import time
import pandas as pd
from ina219 import INA219
import RPi.GPIO as GPIO

# ---- INA219 CONFIGURATION ----
SHUNT_OHMS = 0.1  # Shunt resistor value (default for INA219)

# Initialize INA219 sensors for each source
ina_battery = INA219(SHUNT_OHMS, address=0x40)
ina_solar = INA219(SHUNT_OHMS, address=0x41)
ina_wind = INA219(SHUNT_OHMS, address=0x42)

ina_battery.configure()
ina_solar.configure()
ina_wind.configure()

# ---- CONSTANTS ----
VOLTAGE_BATTERY = 12.0  # Battery voltage in volts
VOLTAGE_SOLAR = 18.0    # Solar panel voltage in volts
VOLTAGE_WIND = 24.0     # Wind turbine voltage in volts

# ---- FUNCTIONS ----
def read_current(ina):
    """Returns the measured current in Amperes."""
    return ina.current() / 1000.0  # Convert from mA to A

def calculate_power(current, voltage):
    """Calculates power in Watts."""
    return current * voltage

# ---- DATA ACQUISITION ----
data = {
    "Time (s)": [],
    "Battery (A)": [],
    "Solar (A)": [],
    "Wind (A)": [],
    "Battery Power (W)": [],
    "Solar Power (W)": [],
    "Wind Power (W)": [],
}

time_interval = 1  # Time interval between measurements in seconds

while True:  # to change if necessary
    current_time = time.time()

    # Read current from each source
    current_battery = read_current(ina_battery)
    current_solar = read_current(ina_solar)
    current_wind = read_current(ina_wind)

    # Calculate power for each source
    power_battery = calculate_power(current_battery, VOLTAGE_BATTERY)
    power_solar = calculate_power(current_solar, VOLTAGE_SOLAR)
    power_wind = calculate_power(current_wind, VOLTAGE_WIND)

    # Record data
    data["Time (s)"].append(current_time)
    data["Battery (A)"].append(current_battery)
    data["Solar (A)"].append(current_solar)
    data["Wind (A)"].append(current_wind)
    data["Battery Power (W)"].append(power_battery)
    data["Solar Power (W)"].append(power_solar)
    data["Wind Power (W)"].append(power_wind)

    print(f"Measurement {i+1}: "
          f"Battery={current_battery:.3f}A, Power={power_battery:.3f}W | "
          f"Solar={current_solar:.3f}A, Power={power_solar:.3f}W | "
          f"Wind={current_wind:.3f}A, Power={power_wind:.3f}W")

    time.sleep(time_interval)  # Pause between measurements

# ---- DATA STORAGE ----
df = pd.DataFrame(data)
print("\nAcquired Data:")
print(df)

# Save data to a CSV file
df.to_csv("power_energy_measurements.csv", index=False)
print("\nData saved to 'power_energy_measurements.csv'.")

GPIO.cleanup()